### Workspace

In [7]:
%matplotlib notebook

from modsim import *

In [8]:
altitude = [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 
            8000, 9000, 10000, 15000, 20000, 25000, 30000,
           40000, 50000, 60000, 70000, 80000]
density = [1.225, 1.112, 1.007, .9093, .8194, .7364, .6601,
          .5900, .5258, .4671, .4135, .1948, .08891, .04008,
          .01841, .003996, .001027, .0003097, .00008283, .000001846]

dd_da = TimeSeries()
for i in range(20):
    x = altitude[i]
    y = density[i]
    dd_da[x] = y

dd_da = interpolate(dd_da, kind='cubic')

In [9]:
condition = Condition(height=90000,
                      thrust_on=200,
                      f_thrust=20,
                      mass=1,
                      length=.1,
                      width=.1,
                      dd_da=dd_da,
                      C_d=1,
                      g=9.8,
                      dt=1)

In [32]:
def make_system(condition):
    unpack(condition)
    
    init = State(height=height, velocity=0)
    
    drag_coef = -C_d*length*width / 2
    
    system = System(init=init,
                    dt=dt,
                    thrust_on=thrust_on,
                    f_thrust=f_thrust,
                    mass=mass,
                    drag_coef=drag_coef,
                    dd_da=dd_da)
    return system

def slope(state, system, dt, count):
    unpack(system)
    height, velocity = state
    
    if (height < thrust_on):
        f_t=f_thrust
    else:
        f_t=0
    
    f_drag = drag_coef * abs(velocity) * velocity * abs(dd_da(height))
    #print(f_drag)
    a_drag = f_drag/mass

    a_thrust = f_t/mass
    a_grav = -g
    a = a_grav + a_thrust + a_drag
    dV = a*dt
    velocity += dV
    
    dH = velocity*dt
    height += dH
         
    return State(height=height, velocity=velocity)

def run_ode_best(system, slope_func):
    count = 0
    unpack(system)
    frame = TimeFrame(columns=init.index)
    
    frame.loc[0] = init
    while(frame.loc[count].velocity <= 0):
        frame.loc[count+dt] = slope_func(frame.loc[count], system, dt, count)
        count += dt
    system.count = count
    system.results = frame
def plot_height(results):
    newfig()
    unpack(results)
    
    plot(height, label='y')
        
    decorate(xlabel='Time (s)',
             ylabel='Position (m)')

    
def plot_velocity(results):
    newfig()
    unpack(results)
    
    plot(velocity, label='v')
    
    decorate(xlabel='Time (s)',
             ylabel='Velocity (m)')
    
def height_at_zero(system):
    unpack(system)
    if(t1 < 0):
        return print("error, t1 is less than 0. William done messed up")
    velocity = system.results.velocity[t1:t2]
    t = interp_inverse(velocity)
    
    time = t(0)
    y = interpolate(system.results.height)
    height = y(time)
    return height

def run_simulation():
    system = make_system(condition)
    run_ode_best(system, slope)
    
    #system.final_height = height_at_zero(system)
    
    return system

In [35]:
condition.set(dt=.01)
condition.set(t_end = 20)
condition.set(height = 100)
condition.set(thrust_on = 5)

#run2 = make_system(condition)
#run_ode_better(run2, slope)
run1 = run_simulation()
print(run1.results)

          height  velocity
0.00  100.000000  0.000000
0.01   99.999020 -0.098000
0.02   99.997060 -0.195999
0.03   99.994120 -0.293997
0.04   99.990200 -0.391992
0.05   99.985300 -0.489983
0.06   99.979421 -0.587968
0.07   99.972561 -0.685947
0.08   99.964722 -0.783918
0.09   99.955903 -0.881881
0.10   99.946105 -0.979834
0.11   99.935327 -1.077776
0.12   99.923570 -1.175705
0.13   99.910834 -1.273621
0.14   99.897119 -1.371523
0.15   99.882424 -1.469409
0.16   99.866752 -1.567278
0.17   99.850100 -1.665129
0.18   99.832471 -1.762961
0.19   99.813863 -1.860772
0.20   99.794277 -1.958562
0.21   99.773714 -2.056329
0.22   99.752173 -2.154073
0.23   99.729656 -2.251791
0.24   99.706161 -2.349484
0.25   99.681689 -2.447149
0.26   99.656241 -2.544786
0.27   99.629817 -2.642393
0.28   99.602418 -2.739969
0.29   99.574043 -2.837514
...          ...       ...
7.29  -36.194639 -2.867217
7.30  -36.222286 -2.764712
7.31  -36.248909 -2.662242
7.32  -36.274507 -2.559807
7.33  -36.299081 -2.457404
7

In [12]:
condition.set(thrust_on = 44)
run2=run_simulation()
run2.final_height

array(15.089428781413437)

In [13]:
#run2.results

In [14]:
#test=run2.results.height[run2.t1:run2.t2]
#newfig()
#plot(test)

In [15]:
#plot_height(run2.results)

In [16]:
#plot_velocity(run1.results)

In [18]:
sweep = SweepSeries()
condition.set(dt=.25)
condition.set(t_end=100)
condition.set(height=1000)
for i in linrange(1, 10, 1):
    condition.set(thrust_on=i)
    system = make_system(condition)
    run_ode_better(system, slope)
    sweep[i] = height_at_zero(system)
   # print(i)
#sweep